# Demo of the LCS package


In [2]:
#preamble
import os, sys
import pandas as pd
import numpy as np
import random
import pickle

## Import Package

In [3]:
# how to import the packaes
from Rulern.LCSModule import LCS        # the core library
from Rulern.RuleModule import Rule      # this is only needed if you create your own rules

## Load Pre-Trained Models (Back-up)

In [4]:
# #how to load the models using pickle

# with open("Eval/LCSvsNN/28072020-bool/"+"0cv_model_LCS.obj", 'rb') as f:
#      model = pickle.load(f)


In [5]:
# #show example rules form a trained model
# # print(b.history)
# for rule in model.rules:
#     if rule.fitness > 1.0:                 # filter out all the bad rules
#         print(rule,rule.fitness)           # print rule and rule fittness

## Generating data (swap with your own data)

In [6]:
# generate data i 0 - 9 are the input bits and o0-4 are the output

# replce this with your own data set and data wrangling operations

# the LCS package can work with dataframes, arrays or numpy arrays

def gen_rand_in_out(arr_len = 10):
    input = []
    for i in range(arr_len):
        input.append(random.choice([1,0]))
    output = np.array(input[0:int(arr_len/2)]) | np.array(input[int(arr_len/2):arr_len]) # logical or of the first and last five bits
    return np.append(input,output)


print(gen_rand_in_out())
df = []
np_samples = 1000
for i in range(np_samples):
    df.append(gen_rand_in_out())

df = pd.DataFrame(np.array(df).reshape(np_samples,15),columns = ["i0","i1","i2","i3","i4","i5","i6","i7","i8","i9","o0","o1","o2","o3","o4"])
print(df)


[1 1 1 1 0 1 1 0 0 1 1 1 1 1 1]
     i0  i1  i2  i3  i4  i5  i6  i7  i8  i9  o0  o1  o2  o3  o4
0     1   0   1   1   1   0   1   0   0   1   1   1   1   1   1
1     1   0   0   1   1   0   1   0   0   0   1   1   0   1   1
2     1   1   1   0   0   1   0   0   1   0   1   1   1   1   0
3     0   1   0   0   1   0   1   0   0   0   0   1   0   0   1
4     1   0   0   0   1   0   1   1   0   0   1   1   1   0   1
..   ..  ..  ..  ..  ..  ..  ..  ..  ..  ..  ..  ..  ..  ..  ..
995   0   0   1   1   1   1   0   1   1   0   1   0   1   1   1
996   1   0   0   0   1   0   1   0   0   0   1   1   0   0   1
997   0   1   1   0   0   1   1   1   0   1   1   1   1   0   1
998   1   1   0   0   0   0   0   0   0   1   1   1   0   0   1
999   0   1   0   0   1   1   0   0   1   1   1   1   0   1   1

[1000 rows x 15 columns]


## Initialise an LCS model (recommended order of operations)

See Appendix B, Table B.1 for a summary of the model parameters

In [7]:
# initialise LCS 

# recommended order of parameter initialisation

def init_LCS():
    lcs = LCS((10,1),(5,1),max_pop = 100)                                 #input and output shapes as well as the max population
    lcs.input_names = ["i0","i1","i2","i3","i4","i5","i6","i7","i8","i9"] # column names of the input 
    lcs.output_names = ["o0","o1","o2","o3","o4"]                         # column names of the outputs
    lcs.initGA()                                                          # initialise genetic algorithms
    lcs.covering_threshold = 5                                            # set a covering threshold - how may rules must match a data instance                                   
    lcs.GA.interval = 0                                                   # the range interval if range antecedents are enabled
    lcs.GA.sigma = 0.0                                                    # sigma of the spread of genetic mutations of the rule values
    lcs.GA.max_ranges = 0                                                 # max number of ranges a rule can have = > i1 > 0.5 and i1 < 1.0
    lcs.GA.max_attribute_comp = 0                                         # max number of attribute comparisons a rule can have = > i0 >= i1
    lcs.GA.max_comp = 1                                                   # max number of attribute comparisons to a cosntant a rule can have = > i0 >= 0.5
    lcs.GA.max_output_attributes = 0                                      # max number of ouput atributes excl. bias => i1*0.5 + i2*0.5 
    lcs.fitness_weights =[1,0,1]                                          # weights on the fitness function c1, c2 and c3 in the report
    lcs.GA.input_template = df[["i0","i1","i2","i3","i4","i5","i6","i7","i8","i9"]].iloc[[0]] # template on an input frame
    lcs.purge_threshold = 1.0                                             # purge threshold
    lcs.type = "Multi-Class"                                              # this by default is "continous" but can be a classifer if it is a a single-classifer
    return lcs

lcs = init_LCS()                                                          # initialise LCS

X_test = df[lcs.input_names]                                              # get input data 

 ## How to add your own rules

In [8]:
rules = []

# how to add manual rules for an or  operation

for i in range(5):
    ant_dict = {
    "i"+str(i):[[0],["=="],[1]]                                          # antecedent dictionary structure
    }
    con_dict = {                                                         # consequent dictionary structure
    "out_var":"o"+str(i),
    "vars":{},
    "bias":1}
    rules.append(Rule("USER"+str(i),ant_dict,con_dict,seq_len = 1))      # name, antecedent, consequent, sequence length (def. 1) 
    
for i in range(5):
    ant_dict = {
    "i"+str(i+5):[[0],["=="],[1]]
    }
    con_dict = {
    "out_var":"o"+str(i),
    "vars":{},
    "bias":1}
    rules.append(Rule("USER"+str(i+5),ant_dict,con_dict,seq_len = 1))

# initalise each rules parameters, if a rule does not have stats, it will not contribute to a classifcation
    
for rule in rules:
    rule.fitness = 2
    rule.correctness = 100
    rule.matched = 100
    rule.evaluated = 100
    lcs.rules.append(rule)

for rule in lcs.rules:
    if rule.fitness > 1.0:                 # filter out all the bad rules
        print(rule)   

If:  i0[0] == 1 Then: o0[1] =  + 1
If:  i1[0] == 1 Then: o1[1] =  + 1
If:  i2[0] == 1 Then: o2[1] =  + 1
If:  i3[0] == 1 Then: o3[1] =  + 1
If:  i4[0] == 1 Then: o4[1] =  + 1
If:  i5[0] == 1 Then: o0[1] =  + 1
If:  i6[0] == 1 Then: o1[1] =  + 1
If:  i7[0] == 1 Then: o2[1] =  + 1
If:  i8[0] == 1 Then: o3[1] =  + 1
If:  i9[0] == 1 Then: o4[1] =  + 1


## Evaluate inputs

In [9]:
# evaluate input data

results,activations = lcs.evaluate_data(X_test)  

### Show results

In [10]:
print(results[0:10].apply(np.ceil).astype("int"),activations[0:10]) #print the prediction and activations for each row

   o0  o1  o2  o3  o4
0   1   1   1   1   1
1   1   1   0   1   1
2   1   1   1   1   0
3   0   1   0   0   1
4   1   1   1   0   1
5   1   0   0   1   1
6   0   1   0   1   1
7   1   1   0   1   1
8   1   1   0   1   1
9   1   1   1   0   1 [6, 4, 5, 3, 4, 4, 3, 4, 5, 6]


In [11]:
y_test= df[lcs.output_names]
print(y_test.iloc[0:10]) # print the true value for comparison

   o0  o1  o2  o3  o4
0   1   1   1   1   1
1   1   1   0   1   1
2   1   1   1   1   0
3   0   1   0   0   1
4   1   1   1   0   1
5   1   0   0   1   1
6   0   1   0   1   1
7   1   1   0   1   1
8   1   1   0   1   1
9   1   1   1   0   1


## How to train your own LCS model

In [12]:
#how to train your own LCS


# initialise new LCS instance
lcs = init_LCS() 


# initialise new LCS instance

lcs.LearningClassifierSystem(X_test.iloc[0:100],y_test.iloc[0:100],mutation_frq = 10,verberose = True,eval = [X_test,y_test],epochs = 10) 

results,activations = lcs.evaluate_data(X_test) 

Epoch:  0  Avg. Fitness:  0.37961713286713283  +/-  0.5731341644763461  # of Rules: 13  acc:  0.011  ttc:  1.3935685000000007  sec
Epoch:  1  Avg. Fitness:  0.9431467290482458  +/-  0.14984297872754176  # of Rules: 9  acc:  0.024  ttc:  2.7810441000000026  sec
Epoch:  2  Avg. Fitness:  0.6082161478617263  +/-  0.543890636952773  # of Rules: 16  acc:  0.314  ttc:  4.332149999999999  sec
Epoch:  3  Avg. Fitness:  0.6423955846552017  +/-  0.4980468762866746  # of Rules: 16  acc:  0.43  ttc:  5.791530199999997  sec
Epoch:  4  Avg. Fitness:  1.020536150735944  +/-  0.024405407779096253  # of Rules: 13  acc:  0.498  ttc:  7.272680499999993  sec
Epoch:  5  Avg. Fitness:  1.023101551388196  +/-  0.04272721369665697  # of Rules: 13  acc:  0.566  ttc:  8.647738799999999  sec
Epoch:  6  Avg. Fitness:  1.0173186984706175  +/-  0.026223537353018384  # of Rules: 12  acc:  0.561  ttc:  10.016231700000006  sec
Epoch:  7  Avg. Fitness:  0.718162051899366  +/-  0.4784746801182186  # of Rules: 23  acc:  

In [13]:
for rule in lcs.rules:
    #if rule.fitness > 0:                 # filter out all the bad rules
    print(rule,rule.fitness)   

If:  i4[0] == 1.0 Then: o4[1] =  + 1.0 1.0304117647058826
If:  i4[0] >= 1.0 Then: o4[1] =  + 1.0 1.03176923076923
If:  i1[0] == 1.0 Then: o1[1] =  + 1.0 1.0050650406504098
If:  i5[0] >= 1.0 Then: o0[1] =  + 1 1.0176666666666592
If:  i6[0] == 1 Then: o1[1] =  + 1 1.0122866817155673
If:  i2[0] >= 1.0 Then: o2[1] =  + 1 1.0096206896551683
If:  i3[0] == 1 Then: o3[1] =  + 1 1.000999999999998
If:  i2[0] == 1 Then: o2[1] =  + 1 1.004194888178912
If:  i1[0] >= 1.0 Then: o1[1] =  + 1 1.0088431372549052
If:  i0[0] == 1 Then: o0[1] =  + 1 1.000999999999999
If:  i0[0] >= 1.0 Then: o0[1] =  + 1 1.0010000000000012
If:  i9[0] == 1 Then: o4[1] =  + 1 1.0010000000000026
If:  i9[0] >= 1.0 Then: o4[1] =  + 1 1.0010000000000003
If:  i5[0] == 1 Then: o0[1] =  + 1 1.0322500000000003
If:  i2[0] == 0 Then: o3[1] =  + 1 0
If:  i2[0] <= 0.0 Then: o2[1] =  + 0 0
If:  i1[0] == 0 Then: o0[1] =  + 0 0
If:  i4[0] >= 0.0 Then: o1[1] =  + 1 0
If:  i3[0] >= 1.0 Then: o2[1] =  + 0 0
If:  i4[0] <= 0.0 Then: o2[1] =  + 0

In [14]:
# show system classfications, recommeded to use ceil for muticlass models outputs
print(results[10:20].apply(np.ceil).astype("int"),activations[0:10]) #print the prediction and activations for each row

    o0  o1  o2  o3  o4
10   1   1   1   0   1
11   1   0   1   0   1
12   1   0   0   0   1
13   1   1   0   1   1
14   1   1   1   0   1
15   1   0   1   0   1
16   1   1   0   1   1
17   0   1   0   0   1
18   1   0   0   1   1
19   1   1   0   0   1 [12, 9, 9, 8, 8, 7, 5, 7, 9, 13]


In [15]:
y_test= df[lcs.output_names]
print(y_test.iloc[10:20]) # print the true value for comparison

    o0  o1  o2  o3  o4
10   1   1   1   1   1
11   1   0   1   1   1
12   1   0   0   0   1
13   1   1   0   1   1
14   1   1   1   0   1
15   1   0   1   1   1
16   1   1   1   1   1
17   0   1   1   1   1
18   1   0   0   1   1
19   1   1   0   0   1
